In [2]:
import math
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import pickle
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction import DictVectorizer

In [1]:
%run 'methods.ipynb'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.33, random_state=0)

features_train, features_test = generateTestTrainFeature(X_train, X_test)
y_train = getY(X_train, labels)
y_test = getY(X_test, labels)


In [ ]:
y_train = getY(X_train, labels)
y_test = getY(X_test, labels)

In [ ]:
#train and test set validation

forest = RandomForestClassifier(criterion='entropy', max_depth=17, n_estimators=50, random_state=0)
forest = forest.fit(features_train , y_train )

In [ ]:
y_pred = forest.predict_proba(features_test)
loss_function(y_test, y_pred)

In [6]:
#k fold validation
kf= KFold(len(y), n_folds=5, shuffle=True, random_state=None)
train_y_list = []
test_y_list = []
train_X_list = []
test_X_list = []
isCache = False
for criterion in ['gini']:
    for n_estimator in [1000]:
        for max_depth in [4, 8, 12, 17, 25, 30, 40, 50]: 
            avg = 0
            i = 0
            for train_index, test_index in kf:
                if not isCache:
                    X_train, X_test = df_train.iloc[train_index] , df_train.iloc[test_index] 
                
                    #features_train, features_test = generateTestTrainFeature(X_train, X_test)
                    
                    features_train = getFeaturesTrain(X_train)
                    #vec_desc, desc_feature = processDescTrain(X_train, True)
                    #upc_dict, upc_vec, upc_feature = getCountVectorTrain(X_train, 'Upc')
                    #fileno_dict, fileno_vec, fileno_feature = getCountVectorTrain(X_train, 'FinelineNumber')
                    
                    features_test = getFeaturesTest(X_test)
                    #desc_feature_test = processDescTest(X_test, True, vec_desc)
                    #upc_feature_test = getCountVectorTest(X_test, 'Upc', upc_dict, upc_vec)
                    #fileno_feature_test = getCountVectorTest(X_test, 'FinelineNumber', fileno_dict, fileno_vec)
                    
                    y_train = getY(X_train, labels)
                    y_test = getY(X_test, labels)
                    
                    train_y_list.append(y_train)
                    train_X_list.append(features_train)
                    test_y_list.append(y_test)
                    test_X_list.append(features_test)
                    
                else:
                    features_train, features_test = train_X_list[i], test_X_list[i]
                    y_train, y_test = train_y_list[i], test_y_list[i]
                    
                forest = RandomForestClassifier(criterion=criterion, max_depth=max_depth, n_estimators=n_estimator, random_state=0) 
                forest.fit(features_train , y_train )
                y_pred = forest.predict_proba(features_test)
                avg += loss_function(y_test, y_pred)
                i += 1    
            isCache = True
            print criterion, n_estimator, max_depth,"=>", avg/len(kf)


gini 1000 4 => 3.33435853318
gini 1000 8 => 3.93321917309
gini 1000 12 => 4.81782253458
gini 1000 17 => 6.24403733144
gini 1000 25 => 7.17762563013
gini 1000 30 => 7.17969905579
gini 1000 40 => 7.17976049814
gini 1000 50 => 7.17976049814


In [5]:
df_train = pd.read_csv('train.csv')
labels, y = getLabeled(df_train['TripType'])
df_test = pd.read_csv('test.csv')

y_train = getY(df_train, labels)

vec_desc, features_train = processDescTrain(X_train, True)

#trained with tuned classifier
forest = RandomForestClassifier(max_depth=17, n_estimators=1000, random_state=0) 
forest.fit(features_train, y_train)
features_test = processDescTest(X_test, True, vec_desc)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [6]:
generateSubmission(features_test, forest, labels, "rf_all")